In [1]:
#Data from:
#https://dados.ons.org.br/dataset/geracao-usina-2

import pandas as pd
import os

#Selecionei a Conj. Bom Jesus
final_df = []
columns = ['GERACAO_CENTRO_GRAVIDADE', 'SIGLA_USINA', 'DATA','PERIODO_COMERCIALIZACAO']
folder = 'data/geracao_ons/'
for file in os.listdir(folder):

    path = os.path.join(folder, file)
    df = pd.read_csv(path, sep=';', usecols=['din_instante', 'val_geracao', 'nom_usina'])
    df = df[df['nom_usina'] == 'Conj. Bom Jesus']
    df['din_instante'] = pd.to_datetime(df['din_instante'])
    df = df.set_index('din_instante').drop(columns=['nom_usina'])
    final_df.append(df)
    
final_df = pd.concat(final_df, axis=0)
final_df.index = pd.to_datetime(final_df.index)
final_df = final_df.rename(columns={'val_geracao':'generation'})
final_df.to_csv('data/ufv_bom_jesus_da_lapa_gen.csv')


In [5]:
daily.describe()

,generation
count,977.000000
mean,14.825193
std,4.443520
min,0.000000
25%,12.347292
50%,15.562458
75%,18.042125
max,23.423458


In [2]:
#Creating a dataframe where the average daily generation is lower than 10 MWh 
final_df_corrected = final_df.copy()

daily = final_df_corrected.resample('D').mean()
date_list = daily[daily['generation']<10].index.date

final_df_corrected['date'] =  pd.to_datetime(final_df_corrected.index.date)

final_df_corrected = final_df_corrected[~(final_df_corrected['date'].isin(date_list))]
final_df_corrected = final_df_corrected.dropna()
final_df_corrected = final_df_corrected.drop(columns=['date'])
final_df_corrected.to_csv('data/ufv_bom_jesus_da_lapa_gen_removed.csv')

C:\Users\u56389\AppData\Local\Temp\1\ipykernel_23988\3516896030.py:9: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  final_df_corrected = final_df_corrected[~(final_df_corrected['date'].isin(date_list))]


In [3]:
#Percentage of the days removed
len(date_list)*24/len(final_df)*100

13.30603889457523